# Differential undervote rate in Lt. Gov contest

Compare undervote rates by mode of voting (paper versus electronic) using hypergeometric 2-sample test.

In [1]:
%matplotlib inline
import math
import numpy as np
import scipy as sp
import scipy.optimize
from scipy.stats import hypergeom, binom, norm, chi2
from scipy import special
from collections import Counter
from cryptorandom.cryptorandom import SHA256
from cryptorandom import sample
from permute.utils import binom_conf_interval
from permute.npc import fisher
import matplotlib.pyplot as plt
import pandas as pd
import csv

In [2]:
statewide_contests = np.array(["Lieutenant Governor", "Secretary Of State", \
                               "Attorney General", "Commissioner Of Agriculture", \
                               "Commissioner Of Insurance", "State School Superintendent", "Commissioner Of Labor", \
                               "Public Service Commission, District 3 - Metro-Atlanta", \
                               "Public Service Commission, District 5 - Western"])
votes = pd.read_csv('../../Data/undervotes_by_county.csv')
votes.head()

,County,Vote type,Total ballots,Contest,Ballots cast,Undervotes
0,Appling,Absentee by Mail,530,Lieutenant Governor,523,7
1,Appling,Advance in Person,3298,Lieutenant Governor,3092,206
2,Appling,Election Day,2978,Lieutenant Governor,2768,210
3,Appling,Provisional,3,Lieutenant Governor,3,0
4,Atkinson,Absentee by Mail,88,Lieutenant Governor,88,0


In [3]:
mode_mask = votes['Vote type'].isin(['Advance in Person', 'Election Day', 'Absentee by Mail'])
dre_mask = votes['Vote type'].isin(['Advance in Person', 'Election Day'])

p_thresh = 0.0001
contests_sig = Counter()

for c in statewide_contests:
    cont_mask = votes['Contest'] == c
    for cty in votes['County'].unique():
        cty_mask = votes['County'] == cty
        N = votes[mode_mask & cont_mask & cty_mask]['Total ballots'].sum()
        G = votes[mode_mask & cont_mask & cty_mask]['Undervotes'].sum()
        g = votes[dre_mask & cont_mask & cty_mask]['Undervotes'].sum()
        n = votes[dre_mask & cont_mask & cty_mask]['Ballots cast'].sum() + g
        # pmf(k, M, n, N) = choose(n, k) * choose(M - n, N - k) / choose(M, N),
        p = 2*min(hypergeom.cdf(g, N, G, n),hypergeom.sf(g-1, N, G, n))
        if p <= p_thresh:
            contests_sig[c] = contests_sig[c]+1
        print(c, cty, N, G, n, g, G/N, g/n, p)

Lieutenant Governor Appling 6806 423 6276 416 0.06215104319717896 0.0662842574888464 2.1439776587662654e-08
Lieutenant Governor Atkinson 2520 218 2432 218 0.0865079365079365 0.08963815789473684 0.0006011988993767051
Lieutenant Governor Bacon 3829 244 3655 242 0.06372420997649517 0.06621067031463748 0.0014452710502285972
Lieutenant Governor Baker 1291 101 1150 95 0.07823392718822618 0.08260869565217391 0.1172912140177249
Lieutenant Governor Baldwin 15633 892 14475 882 0.05705878590161837 0.060932642487046634 6.904218122683707e-19
Lieutenant Governor Banks 6852 220 6629 219 0.03210741389375365 0.033036657112686676 0.01057958862999079
Lieutenant Governor Barrow 27398 1060 26135 1049 0.03868895539820425 0.04013774631719916 3.900219802989691e-11
Lieutenant Governor Bartow 37327 478 35759 476 0.012805743831542852 0.01331133420956962 5.32938619891327e-07
Lieutenant Governor Ben Hill 5533 335 5213 332 0.06054581601301283 0.06368693650489161 4.49526479982811e-06
Lieutenant Governor Berrien 6247

Lieutenant Governor Jefferson 6756 540 6149 531 0.07992895204262877 0.08635550496015612 3.9800612435590653e-13
Lieutenant Governor Jenkins 2856 165 2668 158 0.05777310924369748 0.05922038980509745 0.271929290013697
Lieutenant Governor Johnson 3483 228 3247 217 0.06546080964685616 0.0668309208500154 0.2782267775812352
Lieutenant Governor Jones 12432 494 11648 483 0.03973616473616474 0.04146634615384615 2.87905012617066e-05
Lieutenant Governor Lamar 7346 408 6942 406 0.05554043016607678 0.058484586574474216 3.2800079549672923e-08
Lieutenant Governor Lanier 2679 193 2571 191 0.07204180664427025 0.07429015947102295 0.024928032978287797
Lieutenant Governor Laurens 18939 1045 17653 1024 0.05517714768467184 0.058007137597009005 5.880764614422535e-13
Lieutenant Governor Lee 13549 536 13069 529 0.03956011513764854 0.040477465758665544 0.002238205906476719
Lieutenant Governor Liberty 15358 648 14408 642 0.04219299387941138 0.04455857856746252 1.0462881031104879e-11
Lieutenant Governor Lincoln 39

Secretary Of State Bibb 60460 897 55999 866 0.014836255375454846 0.015464561867176199 9.064081253868293e-07
Secretary Of State Bleckley 4855 124 4598 120 0.02554067971163749 0.02609830361026533 0.4109211038426934
Secretary Of State Brantley 5689 126 5501 124 0.02214800492177887 0.022541356117069623 0.4139150249846568
Secretary Of State Brooks 5696 154 5231 143 0.027036516853932584 0.027337029248709614 0.776808121216839
Secretary Of State Bryan 14970 157 14388 153 0.010487641950567802 0.010633861551292744 0.5291818336362674
Secretary Of State Bulloch 23543 297 22467 280 0.012615214713502952 0.012462723104998442 0.40494706042917417
Secretary Of State Burke 8719 80 7751 75 0.009175364147264595 0.009676170816668817 0.21555379821695064
Secretary Of State Butts 8863 148 8483 141 0.016698634773778632 0.016621478250618885 0.9012343510351347
Secretary Of State Calhoun 1899 40 1714 37 0.021063717746182202 0.021586931155192533 0.8857027787115807
Secretary Of State Camden 17053 238 15860 230 0.013

Secretary Of State Macon 4217 155 3886 149 0.036755987668958975 0.03834276891405044 0.06739790146318843
Secretary Of State Madison 11697 173 11170 169 0.014790117124048902 0.01512981199641898 0.2096126826548903
Secretary Of State Marion 2930 86 2689 82 0.029351535836177476 0.030494607660840462 0.30188111992384153
Secretary Of State McDuffie 8792 130 7924 120 0.014786169244767972 0.01514386673397274 0.5022690042801627
Secretary Of State McIntosh 5408 92 4968 88 0.017011834319526627 0.017713365539452495 0.24075232447687583
Secretary Of State Meriwether 8639 171 8156 166 0.019793957633985414 0.020353114271701816 0.15514326741648682
Secretary Of State Miller 2313 106 2181 101 0.04582792909641158 0.04630903255387437 0.8580788632180962
Secretary Of State Mitchell 7446 139 6882 130 0.018667741069030352 0.01888985759953502 0.7699936294317602
Secretary Of State Monroe 12932 185 12236 185 0.014305598515310857 0.015119320039228506 6.658003128835017e-05
Secretary Of State Montgomery 3528 87 3317 8

Attorney General Cherokee 106299 1697 100987 1639 0.01596440229917497 0.016229811757949044 0.0019240914438680768
Attorney General Clarke 43261 959 40239 912 0.0221677723584753 0.022664579139640647 0.00921420465410043
Attorney General Clay 1186 46 1043 43 0.0387858347386172 0.04122722914669223 0.3465042004226767
Attorney General Clayton 91840 1753 85993 1645 0.019087543554006967 0.019129464026141664 0.7687012680203873
Attorney General Clinch 2254 95 2145 92 0.04214729370008873 0.042890442890442894 0.6293343971869195
Attorney General Cobb 310381 4188 284480 3819 0.013493093971602643 0.01342449381327334 0.28519514116790384
Attorney General Coffee 12595 435 12200 431 0.03453751488685986 0.03532786885245902 0.0036868805279836312
Attorney General Colquitt 12953 385 12439 378 0.029722844128773258 0.030388294879009566 0.02574252425899527
Attorney General Columbia 61591 848 58655 808 0.013768245360523453 0.013775466712130252 1.0289259329632725
Attorney General Cook 5803 115 5559 113 0.019817335

...

Public Service Commission, District 3 - Metro-Atlanta Crisp 7024 263 6661 252 0.03744305239179954 0.03783215733373367 0.57022327175284
Public Service Commission, District 3 - Metro-Atlanta Dade 5450 204 5321 194 0.03743119266055046 0.03645931215936854 0.04423540062189183
Public Service Commission, District 3 - Metro-Atlanta Dawson 11570 180 11131 177 0.015557476231633534 0.0159015362501123 0.17042796741281618
Public Service Commission, District 3 - Metro-Atlanta DeKalb 310968 6618 291296 6149 0.02128193254611407 0.02110911238053389 0.012170672725772132
Public Service Commission, District 3 - Metro-Atlanta Decatur 9043 214 8495 201 0.023664713037708725 0.023660977045320778 1.075979295480614
Public Service Commission, District 3 - Metro-Atlanta Dodge 7051 310 6603 295 0.043965394979435544 0.0446766621232773 0.31694517880674433
Public Service Commission, District 3 - Metro-Atlanta Dooly 3793 147 3597 142 0.038755602425520695 0.03947734222963581 0.4368822628935295
Public Service Commission

Public Service Commission, District 3 - Metro-Atlanta Morgan 9554 208 9144 199 0.02177098597446096 0.021762904636920384 1.0771954633157568
Public Service Commission, District 3 - Metro-Atlanta Murray 11091 313 10826 301 0.028221080155080697 0.02780343617217809 0.14527811825180753
Public Service Commission, District 3 - Metro-Atlanta Muscogee 63141 1165 58719 1094 0.018450768914017833 0.018631107477988386 0.2396873841251743
Public Service Commission, District 3 - Metro-Atlanta Newton 43129 766 40394 724 0.017760671473950242 0.01792345397831361 0.3651655457978468
Public Service Commission, District 3 - Metro-Atlanta Oconee 20733 453 19802 433 0.02184922587179858 0.021866478133521865 1.055579621805487
Public Service Commission, District 3 - Metro-Atlanta Oglethorpe 6484 155 6075 149 0.02390499691548427 0.024526748971193415 0.266783290780498
Public Service Commission, District 3 - Metro-Atlanta Paulding 61222 986 57915 925 0.0161053216164124 0.015971682638349305 0.3040519329955258
Public S

Public Service Commission, District 5 - Western Banks 6852 127 6629 124 0.018534734384121423 0.01870568713229748 0.8047273007916005
Public Service Commission, District 5 - Western Barrow 27398 477 26135 449 0.01741002992919191 0.01718002678400612 0.22940550742667565
Public Service Commission, District 5 - Western Bartow 37327 666 35759 646 0.017842312535162216 0.01806538214155877 0.134702198867361
Public Service Commission, District 5 - Western Ben Hill 5533 109 5213 102 0.019699981926622086 0.01956646844427393 0.8884472670232344
Public Service Commission, District 5 - Western Berrien 6247 148 5925 144 0.023691371858492075 0.02430379746835443 0.22710873920044516
Public Service Commission, District 5 - Western Bibb 60460 1304 55999 1239 0.021567978828977836 0.022125395096341007 0.0005451911597622057
Public Service Commission, District 5 - Western Bleckley 4855 156 4598 148 0.032131822863027806 0.03218790778599391 1.1103478666393036
Public Service Commission, District 5 - Western Brantle

Public Service Commission, District 5 - Western Houston 59158 916 55968 857 0.015483958213597484 0.01531232132647227 0.18432577933394206
Public Service Commission, District 5 - Western Irwin 3556 89 3423 88 0.025028121484814397 0.025708442886356996 0.2928814702730029
Public Service Commission, District 5 - Western Jackson 26889 458 25870 442 0.01703298746699394 0.017085427135678392 0.8604798787546384
Public Service Commission, District 5 - Western Jasper 5907 111 5486 102 0.018791264601320468 0.01859278162595698 0.7885853504297384
Public Service Commission, District 5 - Western Jeff Davis 4815 197 4586 190 0.04091381100726895 0.04143044047099869 0.5419296903754992
Public Service Commission, District 5 - Western Jefferson 6756 272 6149 256 0.04026050917702783 0.041632785818832334 0.0746451471338513
Public Service Commission, District 5 - Western Jenkins 2856 85 2668 78 0.02976190476190476 0.02923538230884558 0.6510448288617523
Public Service Commission, District 5 - Western Johnson 3483

Public Service Commission, District 5 - Western Wilcox 2815 80 2610 74 0.028419182948490232 0.028352490421455937 1.0660068156500393
Public Service Commission, District 5 - Western Wilkes 4371 163 4070 150 0.03729123770304278 0.036855036855036855 0.6621223032847633
Public Service Commission, District 5 - Western Wilkinson 4264 175 3994 168 0.04104127579737336 0.04206309464196294 0.2491133660137237
Public Service Commission, District 5 - Western Worth 7831 198 7526 192 0.025284127186821607 0.025511559925591284 0.6850700503010596


In [4]:
contests_sig

Counter({'Lieutenant Governor': 101,
         'Secretary Of State': 4,
         'Attorney General': 4,
         'Commissioner Of Agriculture': 5,
         'Commissioner Of Insurance': 4,
         'State School Superintendent': 5,
         'Commissioner Of Labor': 2,
         'Public Service Commission, District 3 - Metro-Atlanta': 4,
         'Public Service Commission, District 5 - Western': 4})

In [5]:
# version information
%load_ext version_information
%version_information scipy, numpy, csv, pandas, matplotlib, notebook, cryptorandom, permute

Loading extensions from ~/.ipython/extensions is deprecated. We recommend managing extensions like any other Python packages, in site-packages.


/anaconda/lib/python3.6/site-packages/IPython/core/formatters.py:839: FormatterWarning: JSON expects JSONable list/dict containers, not JSON strings
  FormatterWarning)


Software,Version
Python,3.6.7 64bit [GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
IPython,7.2.0
OS,Darwin 18.2.0 x86_64 i386 64bit
scipy,1.1.0
numpy,1.15.4
csv,1.0
pandas,0.23.1
matplotlib,3.0.2
notebook,5.7.4
cryptorandom,0.2
